## PREPROCESAMIENTO DE DATOS

In [1]:
import pandas as pd

train = pd.read_csv("data/public_train.csv")
print(train.head())

         ccs          adduct   desc_1      desc_2   desc_3     desc_4  \
0  155.71001   Monomer_[M+H] -5.27492   420.85751  0.79886   60.68518   
1  179.56000   Monomer_[M-H] -2.00000  1171.97156  1.13333  121.96276   
2  180.00000  Monomer_[M+Na] -0.70060  1447.22644  1.30616  141.10509   
3  155.53999  Monomer_[M+Na]  0.00000   479.80600  0.92774   74.90678   
4  173.50000   Monomer_[M+H] -1.63669   978.85028  1.24843  112.27611   

    desc_5     desc_6     desc_7    desc_8  ...  fgp_611  fgp_612  fgp_613  \
0  0.22702   54.36304   55.73885   8.28095  ...        0        0        1   
1  0.15520  100.45390   97.68512  11.36667  ...        1        0        0   
2  0.13916  110.47855  108.96603  12.96667  ...        0        1        1   
3  0.21166   65.64590   66.68343   6.83333  ...        1        0        1   
4  0.15789   87.88085   89.68015  10.73333  ...        0        1        0   

   fgp_614  fgp_615  fgp_616  fgp_617  fgp_618  fgp_619  fgp_620  
0        0        1      

In [2]:
import pandas as pd

test = pd.read_csv("data/public_test.csv")
print(test.head())

           adduct   desc_1       desc_2   desc_3     desc_4   desc_5  \
0   Monomer_[M+H] -2.52359    942.17682  1.29512   87.64198  0.16297   
1   Monomer_[M-H]  2.00538  12122.62988  1.80601  621.73560  0.07288   
2   Monomer_[M-H]  2.73984   6543.37451  1.52727  440.95099  0.08565   
3  Monomer_[M+Na] -2.34683    695.94104  1.04344   74.62116  0.17652   
4   Monomer_[M+H]  0.00000    101.73023  0.03813   20.08101  0.35359   

      desc_6     desc_7    desc_8   desc_9  ...  fgp_611  fgp_612  fgp_613  \
0   83.92964   84.34066  11.00000  4.80405  ...        0        1        0   
1  545.80798  554.61530  22.73333  5.64051  ...        0        0        0   
2  305.62857  313.87540  19.90000  5.31395  ...        1        1        1   
3   65.28460   63.78675  10.81429  4.25346  ...        1        0        0   
4   19.35335   20.59024   6.06667  2.62874  ...        1        1        0   

   fgp_614  fgp_615  fgp_616  fgp_617  fgp_618  fgp_619  fgp_620  
0        1        1        1   

Ya visaulizados los datos, se procede a realizar el preprocesamiento de los mismos. Para ello, se realizarán las siguientes tareas:
- Eliminación de columnas innecesarias(Ingeniería de características)
- Imputación
- Codificación de variables categóricas
- Normalización de los datos


In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
#Codificación de la columna adduct: Convierte la columna categórica adduct en variables numéricas usando técnicas como One-Hot Encoding o Label Encoding.


encoder = OneHotEncoder(sparse_output = False)
adduct_encoded = encoder.fit_transform(train[['adduct']])
train_encoded = pd.concat([train.drop(columns=['adduct']), pd.DataFrame(adduct_encoded)], axis=1)
scaler = RobustScaler()
X_train = scaler.fit_transform(train_encoded.drop(columns=['ccs']).values)


"""
scaler = StandardScaler()#para las caracteristicas numericas
numeric_imputer = SimpleImputer(strategy="mean")#Para los nans de caracteristicas numericas
categorical_imputer = SimpleImputer(strategy="most_frequent")# Para los nans de caracteristicas categoricas
oh_encoder = OneHotEncoder(sparse_output=False)# para las caracteristicas categoricas

numeric_transformer = Pipeline(steps=[
    ("imputer", numeric_imputer),
    ("scaler", scaler)
])

categorical_transformer = Pipeline(steps=[
    ("imputer", categorical_imputer),
    ("encoder", oh_encoder)
])


transformer = ColumnTransformer(
    transformers=[
        ('numeric_imp', numeric_transformer, ['Age', 'FamSize', 'Fare']),  
        ('categorial_imp', categorical_transformer, ['Pclass', 'Sex', 'Embarked']) 
    ]
)

transformer = transformer.set_output(transform="pandas")
"""








'\nscaler = StandardScaler()#para las caracteristicas numericas\nnumeric_imputer = SimpleImputer(strategy="mean")#Para los nans de caracteristicas numericas\ncategorical_imputer = SimpleImputer(strategy="most_frequent")# Para los nans de caracteristicas categoricas\noh_encoder = OneHotEncoder(sparse_output=False)# para las caracteristicas categoricas\n\nnumeric_transformer = Pipeline(steps=[\n    ("imputer", numeric_imputer),\n    ("scaler", scaler)\n])\n\ncategorical_transformer = Pipeline(steps=[\n    ("imputer", categorical_imputer),\n    ("encoder", oh_encoder)\n])\n\n\ntransformer = ColumnTransformer(\n    transformers=[\n        (\'numeric_imp\', numeric_transformer, [\'Age\', \'FamSize\', \'Fare\']),  \n        (\'categorial_imp\', categorical_transformer, [\'Pclass\', \'Sex\', \'Embarked\']) \n    ]\n)\n\ntransformer = transformer.set_output(transform="pandas")\n'

## ENTRENAMIENTO Y ESTIMACIÓN DE ERROR

Después de realizar el preprocesamiento de los datos, se procederá a entrenar un modelo de regresión logística y a estimar el error del modelo. Para ello, se realizarán las siguientes tareas:
- División de los datos en conjuntos de entrenamiento y prueba
- Entrenamiento del modelo
- Estimación del error del modelo

Usamos un pipeline para el preprocesamiento, selección de características y entrenamiento del modelo.
Además , hacemos uso de la técnica de validación cruzada K-fold reduciendo el riesgo de sobreajuste debido a la división de los datos en conjuntos de entrenamiento y prueba.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import median_absolute_error , make_scorer
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline 
from sklearn.impute import SimpleImputer 
import pandas as pd
import numpy as np  # Necesario para calcular el promedio de los errores

# Asegúrate de que los nombres de las columnas sean cadenas
X = train_encoded.drop(columns=['ccs'])
y = train_encoded['ccs']

# Asegurar que X sea un DataFrame con columnas como cadenas
X.columns = X.columns.astype(str)

# Definir el pipeline mejorado
pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('feature_selection', SelectFromModel(Lasso(alpha=0.005, random_state=42), threshold='median')),
    ('model', GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=3, random_state=42))  # Modelo optimizado
])

# Definir el scorer para MDAE
scorer = make_scorer(median_absolute_error, greater_is_better=False)

# Realizar validación cruzada con KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Aumentado a 5 pliegues

# Lista para almacenar los errores en cada pliegue
cv_errors = []

# Realizar validación cruzada
for train_index, val_index in kf.split(X, y):
    # Dividir los datos en entrenamiento y validación para cada pliegue
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Entrenamiento del pipeline
    pipeline.fit(X_train, y_train)

    # Predicción en el conjunto de validación
    y_pred = pipeline.predict(X_val)

    # Calcular MDAE en el conjunto de validación
    medae = median_absolute_error(y_val, y_pred)
    cv_errors.append(medae)

# Promedio de los errores de validación cruzada
average_medae = np.mean(cv_errors)
print("Promedio MDAE en validación cruzada:", average_medae)

# Entrenamiento del pipeline en todo el conjunto de entrenamiento
pipeline.fit(X, y)

# Predicción y evaluación final
final_predictions = pipeline.predict(X_val)
final_medae = median_absolute_error(y_val, final_predictions)
print("MDAE en conjunto de validación final:", final_medae)




TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'SMOTE(random_state=42)' (type <class 'imblearn.over_sampling._smote.base.SMOTE'>) doesn't

## GENERACIÓN DE PREDICCIONES

In [6]:


# Paso 3: Generación de predicciones

# Codificar la columna "adduct" en el conjunto de prueba con el mismo encoder
adduct_test_encoded = encoder.transform(test[['adduct']])
adduct_test_encoded = pd.DataFrame(adduct_test_encoded, columns=encoder.get_feature_names_out(['adduct']))

# Combinar el conjunto de prueba con las columnas codificadas
test_encoded = pd.concat([test.drop(columns=['adduct']), adduct_test_encoded], axis=1)

# Alinear las columnas de test_encoded con las de X_train
test_encoded = test_encoded.reindex(columns=X.columns, fill_value=0)

# Rellenar valores nulos
test_encoded = test_encoded.fillna(test_encoded.mean())

# Normalizar las características del conjunto de prueba con el mismo escalador
X_test = scaler.transform(test_encoded)

# Generar predicciones en el conjunto de prueba
test_preds = pipeline.predict(X_test)

# Guardar las predicciones en un archivo CSV
pd.DataFrame(test_preds).to_csv("test_preds.csv", index=False, header=False)




c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


## CONCLUSIONES Y JUSTIFICACIÓN

En el inicio, convertirmos la columna categórica adduct en una representación numérica utilizando One-Hot Encoding.
¿Por qué?, los modelos que estamos usando no trabajan con datos categóricos es decir "letras", necesitan que estas columnas sean transformadas en valores numéricos. Posteriormente, escalamos los datos para que todas las características tengan la misma importancia en el modelo usando RobustScaler ya que  es robusto frente a valores atípicos, es decir, no se ve afectado por valores extremos.Dentro del pipeline, se incluyó la selección de características utilizando el método de imputación de características estandar, que rellena los valores faltantes con la media de la columna, seguido de esto , se realiza la reguralización L1 para seleccionar las características más importantes y redciendo a 0 las menos importantes.¿Porque no usamos L2?, porque L1 es más efectiva, ya que reduce a 0 las menos importantes, mientras que L2 solo reduce su peso manteniendo TODAS las caracteristicas y finalmente en el pipeline se incluyó el modelo de regresión logística Gradient Boosting que trata de mejorar el rendimiento de los modelos de regresión logística mediante la combinación de varios modelos de regresión logística. Finalmente, se estimó el error del modelo utilizando la validación cruzada K-fold, que reduce el riesgo de sobreajuste debido a la división de los datos en conjuntos de entrenamiento y prueba. 

El modelo de regresión logística tiene un rendimiento aceptable para la predicción de la variable objetivo. Sin embargo, mejoraria el rendimiento del modelo mediante la optimización de los hiperparámetros del modelo, esto se implementó, sin embargo , la espera de tiempo de ejecución era muy larga, por lo que se decidió no incluirlo en el notebook.